In [ ]:
import Tokenizer
import Translator

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow as tf

# Load the dataset

In [ ]:
examples, metadata = tfds.load('ted_hrlr_translate/he_to_pt',
                               with_info=True,
                               as_supervised=True)

train_examples, test_examples = examples['train'], examples['validation']
test_examples

In [ ]:
metadata

# Set the tokenizers

## Train new tokenizers

In [ ]:
tokenizer_he = Tokenizer.Tokenizer(train_examples.map(lambda he, pt: he))
tokenizer_pt = Tokenizer.Tokenizer(train_examples.map(lambda he, pt: pt))

### Save the tokenizers for reload

In [ ]:
tokenizer_he.save('tokenizer_he')
tokenizer_pt.save('tokenizer_pt')

## Info about the tokenized Data

In [ ]:
lengths = []

for he_examples, pt_examples in train_examples.batch(1024):
  he_tokens = tokenizer_he.tokenize(he_examples)
  lengths.append(he_tokens.row_lengths())

  pt_tokens = tokenizer_pt.tokenize(pt_examples)
  lengths.append(pt_tokens.row_lengths())
  print('.', end='', flush=True)

In [ ]:
all_lengths = np.concatenate(lengths)

plt.hist(all_lengths, np.linspace(0, 200, 101))
plt.ylim(plt.ylim())
max_length = max(all_lengths)
plt.plot([max_length, max_length], plt.ylim())
plt.title(f'Maximum tokens per example: {max_length}');
plt.xlabel('Tokens Per Example')
plt.ylabel('Exampels')

## Preper the batches

In [ ]:
# Create training and validation set batches.
test_batches = Translator.make_batches(test_examples, tokenizer_he, tokenizer_pt)
train_batches = Translator.make_batches(train_examples, tokenizer_he, tokenizer_pt)

# Create The Transformer

The prameters of the model

In [ ]:
num_layers = 6
d_model = 256
dff = 2028
num_heads = 8
dropout_rate = 0.1

In [ ]:
transformer = Translator.Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=tokenizer_he.word_count(),
    target_vocab_size=tokenizer_pt.word_count(),
    dropout_rate=dropout_rate)

Run one batch to build the model

In [ ]:
for (he, pt), _ in test_batches.take(1):
    transformer((he,pt))

In [ ]:
transformer.summary()

Set the optimizer and the CustomSchedule

In [ ]:
learning_rate = Translator.CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [ ]:
transformer.compile(
    loss=Translator.masked_loss,
    optimizer=optimizer,
    metrics=[Translator.masked_accuracy])

## Fit the model

In [ ]:
NUM_EPOCHS = 20

In [ ]:
transformer.fit(train_batches,
                epochs=NUM_EPOCHS,
                validation_data=test_batches)

## Save the weights

In [ ]:
transformer.save_weights(f'translator-dff:{dff} d_model:{d_model} layers{num_layers}.weights.h5')

# Load a Pre-Trained model

In [ ]:
import Tokenizer
import Translator
import tensorflow as tf

## Load the tokenizers

In [ ]:
tokenizer_he = Tokenizer.Tokenizer('Saved/tokenizer_he.pkl', isLoad= True)
tokenizer_pt = Tokenizer.Tokenizer('Saved/tokenizer_pt.pkl', isLoad= True)

## Load the transformer
The parameters of the model should match the parameters of the saved model

In [ ]:
num_layers = 6
d_model = 256
dff = 2028
num_heads = 8
dropout_rate = 0.1

In [ ]:
transformer = Translator.Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=tokenizer_he.word_count(),
    target_vocab_size=tokenizer_pt.word_count(),
    dropout_rate=dropout_rate)

Run on empty tensors to build the model

In [ ]:
x = tf.constant([[0]*40]*40)
transformer((x,x))
pass

In [ ]:
transformer.load_weights('Saved/transformer.weights.h5')

In [ ]:
transformer.summary()

# Use the model

Set the translator Interface

In [ ]:
translator = Translator.Translator(tokenizer_he,tokenizer_pt,transformer)

In [ ]:
sentence = 'ילדים אוהבים כיסאות.'
ground_truth = 'As crianças adoram cadeiras.'

translated_text, translated_tokens, attention_weights = translator(
    tf.constant(sentence))
Translator.print_translation(sentence, translated_text, ground_truth)

Plot of one Head

In [ ]:
head = 0
# Shape: `(batch=1, num_heads, seq_len_q, seq_len_k)`.
attention_heads = tf.squeeze(attention_weights, 0)
attention = attention_heads[head]
attention.shape

In [ ]:
in_tokens = tf.convert_to_tensor([sentence])
in_tokens = tokenizer_he.tokenize(in_tokens).to_tensor()
in_tokens = tokenizer_he.lookup(in_tokens)[0]
in_tokens

In [ ]:
Translator.plot_attention_head(in_tokens, translated_tokens, attention)

Plot of all heads

In [ ]:
Translator.plot_attention_weights(sentence,
                       translated_tokens,
                       attention_weights[0],
                       tokenizer_he)

## More Exampels

In [ ]:
sentence = 'אבא שלי הכי חזק מאחר והוא ניצח את התנין האיום.'
ground_truth = 'Meu pai é o mais forte desde que derrotou o terrível crocodilo.'

translated_text, translated_tokens, attention_weights = translator(
    tf.constant(sentence))
Translator.print_translation(sentence, translated_text, ground_truth)
Translator.plot_attention_weights(sentence,
                       translated_tokens,
                       attention_weights[0],
                       tokenizer_he)

In [ ]:
sentence = 'יום יבוא ואסיים את הפרויקט הזה.'
ground_truth = 'Um dia vou terminar esse projeto.'

translated_text, translated_tokens, attention_weights = translator(
    tf.constant(sentence))
Translator.print_translation(sentence, translated_text, ground_truth)
Translator.plot_attention_weights(sentence,
                       translated_tokens,
                       attention_weights[0],
                       tokenizer_he)

In [ ]:
sentence = 'זה היה קשה, אך לבסוף הצלחנו'
ground_truth = 'Foi difícil, mas finalmente conseguimos'

translated_text, translated_tokens, attention_weights = translator(
    tf.constant(sentence))
Translator.print_translation(sentence, translated_text, ground_truth)
Translator.plot_attention_weights(sentence,
                       translated_tokens,
                       attention_weights[0],
                       tokenizer_he)